In [1]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

#import the packages
from newspaper import Article
import random
import string
import numpy as np
import warnings
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:

#ignore the warnings
warnings.filterwarnings('ignore')
     

In [5]:
#download package from nltk
nltk.download('punkt',quiet=True)
nltk.download('wordnet',quiet=True)


True

In [6]:

#get artical url
article= Article('https://en.wikipedia.org/wiki/Cleopatra')
article.download()
article.parse()
article.nlp()
corpus=article.text
#print
print(corpus)

Queen of Egypt from 51 to 30 BC

Cleopatra VII Philopator (Koinē Greek: Κλεοπάτρα Φιλοπάτωρ, "Cleopatra the father-beloved"; 70/69 BC – 10 August 30 BC) was Queen of the Ptolemaic Kingdom of Egypt from 51 to 30 BC, and its last active ruler.[note 5] A member of the Ptolemaic dynasty, she was a descendant of its founder Ptolemy I Soter, a Macedonian Greek general and companion of Alexander the Great.[note 6] After the death of Cleopatra, Egypt became a province of the Roman Empire, marking the end of the last Hellenistic state in the Mediterranean and of the age that had lasted since the reign of Alexander (336–323 BC).[note 7] Although her first language was Koine Greek, she was the only Ptolemaic ruler to learn and use the Egyptian language.[note 8]

In 58 BC, Cleopatra presumably accompanied her father, Ptolemy XII Auletes, during his exile to Rome after a revolt in Egypt (a Roman client state) allowed his rival daughter Berenice IV to claim his throne. Berenice was killed in 55 BC w

In [7]:

#tokenization
text=corpus
sent_tokens=nltk.sent_tokenize(text)
print(sent_tokens)


['Queen of Egypt from 51 to 30 BC\n\nCleopatra VII Philopator (Koinē Greek: Κλεοπάτρα Φιλοπάτωρ, "Cleopatra the father-beloved"; 70/69 BC – 10 August 30 BC) was Queen of the Ptolemaic Kingdom of Egypt from 51 to 30 BC, and its last active ruler.', '[note 5] A member of the Ptolemaic dynasty, she was a descendant of its founder Ptolemy I Soter, a Macedonian Greek general and companion of Alexander the Great.', '[note 6] After the death of Cleopatra, Egypt became a province of the Roman Empire, marking the end of the last Hellenistic state in the Mediterranean and of the age that had lasted since the reign of Alexander (336–323 BC).', '[note 7] Although her first language was Koine Greek, she was the only Ptolemaic ruler to learn and use the Egyptian language.', '[note 8]\n\nIn 58 BC, Cleopatra presumably accompanied her father, Ptolemy XII Auletes, during his exile to Rome after a revolt in Egypt (a Roman client state) allowed his rival daughter Berenice IV to claim his throne.', 'Beren

In [8]:

#creating a dictionary to remove the punctuation
remove_punct_dict=dict( (ord(punct),None) for punct in string.punctuation)
print(string.punctuation)
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [9]:
#create a function to return lower case words 
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))
print(LemNormalize(text))


['queen', 'of', 'egypt', 'from', '51', 'to', '30', 'bc', 'cleopatra', 'vii', 'philopator', 'koinē', 'greek', 'κλεοπάτρα', 'φιλοπάτωρ', 'cleopatra', 'the', 'fatherbeloved', '7069', 'bc', '–', '10', 'august', '30', 'bc', 'was', 'queen', 'of', 'the', 'ptolemaic', 'kingdom', 'of', 'egypt', 'from', '51', 'to', '30', 'bc', 'and', 'its', 'last', 'active', 'rulernote', '5', 'a', 'member', 'of', 'the', 'ptolemaic', 'dynasty', 'she', 'was', 'a', 'descendant', 'of', 'its', 'founder', 'ptolemy', 'i', 'soter', 'a', 'macedonian', 'greek', 'general', 'and', 'companion', 'of', 'alexander', 'the', 'greatnote', '6', 'after', 'the', 'death', 'of', 'cleopatra', 'egypt', 'became', 'a', 'province', 'of', 'the', 'roman', 'empire', 'marking', 'the', 'end', 'of', 'the', 'last', 'hellenistic', 'state', 'in', 'the', 'mediterranean', 'and', 'of', 'the', 'age', 'that', 'had', 'lasted', 'since', 'the', 'reign', 'of', 'alexander', '336–323', 'bcnote', '7', 'although', 'her', 'first', 'language', 'was', 'koine', 'gre

In [10]:

#keywords for greetings
greeting_input=["hi","hello","hey","hola"]
greeting_response=["howdy","hey there","hi","hello :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)

In [11]:
def response(user_response):
 #user response and robo responce
  #user_response="WHat is chronic disease"
  user_response=user_response.lower()
  #print(user_response)
  #robo response
  robo_response=''
  sent_tokens.append(user_response)
  #print(sent_tokens)
  tfidfvec=TfidfVectorizer(tokenizer=LemNormalize , stop_words='english')
  tfidf=tfidfvec.fit_transform(sent_tokens)
  #print(tfidf)
  #get similarity score
  val=cosine_similarity(tfidf[-1],tfidf)
  #print(val)
  idx=val.argsort()[0][-2]
  flat=val.flatten()
  flat.sort()
  score=flat[-2]
  #print(score)
  if score==0:
    robo_response=robo_response+"sorry,i dont understand"
  else:
    robo_response=robo_response+sent_tokens[idx]

  sent_tokens.remove(user_response)
  return robo_response

In [ ]:
greeting_input=["hi","hello","hey","hola"]
greeting_response=["howdy","hey there","hi","hello :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)
flag=True
print("hello!!! this is Rudra,i can answer your queris related to our comapny ,type bye to exit")
while(flag==True):
  user_response=input("user:")
  #user_response=user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("Rudra: anytime :)")
    else:
       if( greeting(user_response) != None):
         print("Rudra: "+ greeting(user_response))
       else:
         print("Rudra:"+response(user_response))
  else:
    flag=False
    print("Rudra: see you later :)")

hello!!! this is Rudra,i can answer your queris related to our comapny ,type bye to exit
user:who is cleopatra
Rudra:Hollywood films of the 20th century were influenced by earlier Victorian media, which helped to shape the character of Cleopatra played by Theda Bara in Cleopatra (1917), Claudette Colbert in Cleopatra (1934), and Elizabeth Taylor in Cleopatra (1963).
user:where cleopatra lived
Rudra:Plutarch lived a century after Cleopatra but relied on primary sources, such as Philotas of Amphissa, who had access to the Ptolemaic royal palace, Cleopatra's personal physician named Olympos, and Quintus Dellius, a close confidant of Mark Antony and Cleopatra.
user:cleopatra was queen of which country
Rudra:After their meeting at Tarsos in 41 BC, the queen had an affair with Antony.
user:who is Antony
Rudra:Relationship with Mark Antony

Cleopatra invited Antony to come to Egypt before departing from Tarsos, which led Antony to visit Alexandria by November 41 BC.
